This notebook is licensed under the MIT License.

MIT License

Copyright (c) 2023 Michael Tullius

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

<h1>Analyze Serum Antibody</h1>

<h2>Import Libraries and Display Versions</h2>

In [1]:
import datetime
from enum import auto, Enum
import os
import pathlib
import sys

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import altair as alt
  
import scipy as sp
from scipy import interpolate

from IPython.display import display, HTML

In [2]:
now = datetime.datetime.now()
todays_date = str(now.date())
print(f"Date: {now.strftime('%Y-%m-%d %H:%M')}")

Date: 2023-02-27 10:36


In [3]:
print("Python version: %s\n" % sys.version)

Python version: 3.11.0 | packaged by conda-forge | (main, Oct 25 2022, 06:12:32) [MSC v.1929 64 bit (AMD64)]



In [4]:
pd.show_versions()

C:\Users\mitullius\.conda\envs\py311\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")



INSTALLED VERSIONS
------------------
commit           : 2e218d10984e9919f0296931d92ea851c6a6faf5
python           : 3.11.0.final.0
python-bits      : 64
OS               : Windows
OS-release       : 10
Version          : 10.0.19044
machine          : AMD64
processor        : Intel64 Family 6 Model 165 Stepping 5, GenuineIntel
byteorder        : little
LC_ALL           : None
LANG             : None
LOCALE           : English_United States.1252

pandas           : 1.5.3
numpy            : 1.24.2
pytz             : 2022.7.1
dateutil         : 2.8.2
setuptools       : 67.3.1
pip              : 23.0
Cython           : None
pytest           : None
hypothesis       : None
sphinx           : None
blosc            : None
feather          : None
xlsxwriter       : None
lxml.etree       : None
html5lib         : None
pymysql          : None
psycopg2         : None
jinja2           : 3.1.2
IPython          : 8.10.0
pandas_datareader: None
bs4              : 4.11.2
bottleneck       : None
brotli

# Edit this cell to incude your input directory and input file, which contains all of the parameters for the experiment.

## The input file must have the following sheets:

### The 'Directories' sheet contains the names of Directories and Files.
### The 'Conditions' sheet contains all the necessary information regarding the content of the wells of all 96-well plates used in this experiment.
### The 'Plates' sheet contains the file names for the raw data for the 96-well plates.
### The 'Dilutions' sheet contains the fold-dilution values.

In [5]:
class ExperimentName(Enum):
    EXAMPLE = auto()
    Bp_m01 = auto()
    Bp_m02 = auto()

class Experiment:
    
    def __init__(self, name, input_directory, input_file_name):
        
        self.name = name
        self.input_directory = input_directory
        self.input_file_name = input_file_name
        
experiments = {}
 
experiments[ExperimentName.EXAMPLE] = Experiment(ExperimentName.EXAMPLE,
                                                 './example',
                                                 'example input.xlsx')

experiments[ExperimentName.Bp_m01] = Experiment(ExperimentName.Bp_m01,
                                                r'C:\Users\mitullius\Notebook\2023\Bp vaccine paper\Re-analysis of Serum Ab\Bp_m01',
                                                'Bp m01 serum antibody - plate maps 2021 05-05.xlsx')

experiments[ExperimentName.Bp_m02] = Experiment(ExperimentName.Bp_m02,
                                                r'C:\Users\mitullius\Notebook\2023\Bp vaccine paper\Re-analysis of Serum Ab\Bp_m02',
                                                'Bp m02 serum antibody - plate maps 2021 03-22.xlsx')


# Select the desired experiment for processing

In [6]:
exp = experiments[ExperimentName.EXAMPLE]

EXPERIMENT_NAME = exp.name
INPUT_DIRECTORY = exp.input_directory
INPUT_FILE_NAME = exp.input_file_name

CUTOFF = 0.05

print(f'\n{EXPERIMENT_NAME=}\n{INPUT_DIRECTORY=}\n{INPUT_FILE_NAME=}\n{CUTOFF=}\n')


EXPERIMENT_NAME=<ExperimentName.EXAMPLE: 1>
INPUT_DIRECTORY='./example'
INPUT_FILE_NAME='example input.xlsx'
CUTOFF=0.05



In [7]:
python_path = pathlib.Path(os.getcwd())
input_path = pathlib.Path(INPUT_DIRECTORY).absolute()

print('\n')
print(f'Python notebook directory:\t{python_path}')
print(f'Input directory:\t\t{input_path}')
print('\n')



Python notebook directory:	C:\Users\mitullius\Notebook\2023\Python\serum-antibody-ELISA
Input directory:		C:\Users\mitullius\Notebook\2023\Python\serum-antibody-ELISA\example




In [8]:
directories = (
    pd.read_excel(input_path / INPUT_FILE_NAME, sheet_name='Directories')
    .set_index('Key')
    .to_dict()['Value']
)
directories

{'Raw Data Directory': './raw data',
 'Output Directory': './output {DATE}',
 'Output File': 'EXAMPLE serum antibody - Data Formatted for GraphPad Prism {DATE}.xlsx',
 'Plates File': 'EXAMPLE serum antibody - All Plates {DATE}.xlsx'}

In [9]:
os.chdir(input_path)

raw_data_path = pathlib.Path(directories['Raw Data Directory']).absolute()

output_dir = directories['Output Directory']
output_dir = output_dir.replace('{DATE}', todays_date)
output_path = pathlib.Path(output_dir).absolute()
output_path.mkdir(parents=True, exist_ok=True)

output_file = directories['Output File']
output_file = output_file.replace('{DATE}', todays_date)

plates_file = directories['Plates File']
plates_file = plates_file.replace('{DATE}', todays_date)

os.chdir(python_path)

print('\n')
print(f'{raw_data_path=}')
print(f'{output_path=}')
print(f'{output_file=}')
print(f'{plates_file=}')
print('\n')



raw_data_path=WindowsPath('C:/Users/mitullius/Notebook/2023/Python/serum-antibody-ELISA/example/raw data')
output_path=WindowsPath('C:/Users/mitullius/Notebook/2023/Python/serum-antibody-ELISA/example/output 2023-02-27')
output_file='EXAMPLE serum antibody - Data Formatted for GraphPad Prism 2023-02-27.xlsx'
plates_file='EXAMPLE serum antibody - All Plates 2023-02-27.xlsx'




In [10]:
writer = pd.ExcelWriter(output_path / output_file)
plates_writer = pd.ExcelWriter(output_path / plates_file)

# DataFrame Helper Functions

In [11]:
def display_df(input_df, name):
    
    display(HTML(f'<h1>{name}</h1><br>'))
    display(input_df)
    display(HTML('<br>'))
    return input_df

def display_df_and_write_to_excel(input_df, writer, sheet_name):
    
    display(HTML(f'<h1>{sheet_name}</h1><br>'))
    display(input_df)
    display(HTML('<br>'))
    input_df.to_excel(writer, sheet_name=sheet_name)
    return input_df

<h2>Load Experimental Details from Excel File</h2>

In [12]:
name = 'Conditions'

df = (
    pd.read_excel(input_path / INPUT_FILE_NAME, sheet_name=name)
    .fillna(value='None')
    .pipe(display_df_and_write_to_excel, writer, sheet_name=name)
)

,Group,#,Mouse,Plate,Dilution_1,Dilution_2,Dilution_3,Dilution_4,Dilution_5,Dilution_6
0,A,1,A1,AB1,A1,A2,A3,A4,A5,A6
1,A,2,A2,AB1,B1,B2,B3,B4,B5,B6
2,A,3,A3,AB1,C1,C2,C3,C4,C5,C6
3,A,4,A4,AB1,D1,D2,D3,D4,D5,D6
4,A,5,A5,AB1,E1,E2,E3,E4,E5,E6
...,...,...,...,...,...,...,...,...,...,...
123,H,4,H4,GH2,E12,E11,E10,E9,E8,E7
124,H,5,H5,GH2,D12,D11,D10,D9,D8,D7
125,H,6,H6,GH2,C12,C11,C10,C9,C8,C7
126,H,7,H7,GH2,B12,B11,B10,B9,B8,B7


## Show unique parameters

In [13]:
columns = ['Plate', 'Group', 'Mouse']

for column in columns:
    if column == 'Mouse':
        name = f'Unique Mice'
    else:
        name = f'Unique {column}s'

    series = df[column]
    (
        pd.DataFrame(series.unique(), columns=[series.name])
        .pipe(display_df_and_write_to_excel, writer, sheet_name=name)
    )

,Plate
0,AB1
1,AB2
2,CD1
3,CD2
4,EF1
5,EF2
6,GH1
7,GH2


,Group
0,A
1,B
2,C
3,D
4,E
5,F
6,G
7,H


,Mouse
0,A1
1,A2
2,A3
3,A4
4,A5
...,...
59,H4
60,H5
61,H6
62,H7


In [14]:
name = 'Plates'

plates_sheet_df = (
    pd.read_excel(input_path/INPUT_FILE_NAME, sheet_name=name)
    .set_index(['Plate'])
    .fillna(value='None')
    .pipe(display_df_and_write_to_excel, writer, sheet_name=name)
)

,File,Group,Antigen,1st Ab,2nd Ab,Condition
Plate,,,,,,
AB1,Plate AB1 (Serum Ab) 2021 05-20.xls,A and B,Antigen 1,serum,IgG,"Group A and B, Antigen: Antigen 1, 1st Ab: ser..."
AB2,Plate AB2 (Serum Ab) 2021 05-20.xls,A and B,Antigen 2,serum,IgG,"Group A and B, Antigen: Antigen 2, 1st Ab: ser..."
CD1,Plate CD1 (Serum Ab) 2021 05-20.xls,C and D,Antigen 1,serum,IgG,"Group C and D, Antigen: Antigen 1, 1st Ab: ser..."
CD2,Plate CD2 (Serum Ab) 2021 05-20.xls,C and D,Antigen 2,serum,IgG,"Group C and D, Antigen: Antigen 2, 1st Ab: ser..."
EF1,Plate EF1 (Serum Ab) 2021 05-20.xls,E and F,Antigen 1,serum,IgG,"Group E and F, Antigen: Antigen 1, 1st Ab: ser..."
EF2,Plate EF2 (Serum Ab) 2021 05-20.xls,E and F,Antigen 2,serum,IgG,"Group E and F, Antigen: Antigen 2, 1st Ab: ser..."
GH1,Plate GH1 (Serum Ab) 2021 05-20.xls,G and H,Antigen 1,serum,IgG,"Group G and H, Antigen: Antigen 1, 1st Ab: ser..."
GH2,Plate GH2 (Serum Ab) 2021 05-20.xls,G and H,Antigen 2,serum,IgG,"Group G and H, Antigen: Antigen 2, 1st Ab: ser..."


In [15]:
name = 'Unique Antigens'

series = plates_sheet_df['Antigen']
(
    pd.DataFrame(series.unique(), columns=[series.name])
    .pipe(display_df_and_write_to_excel, writer, sheet_name=name)
);

,Antigen
0,Antigen 1
1,Antigen 2


In [16]:
name = 'Dilutions'

dilutions_sheet_df = (
    pd.read_excel(input_path / INPUT_FILE_NAME, sheet_name=name)
    .set_index(['Dilution_Name'])
    .fillna(value='None')
    .pipe(display_df_and_write_to_excel, writer, sheet_name=name)
)

,Dilution
Dilution_Name,
Dilution_1,200
Dilution_2,800
Dilution_3,3200
Dilution_4,12800
Dilution_5,51200
Dilution_6,204800


<h2>Function for loading the raw data (Excel format) from a 96-well plate (A415 and A750) into a DataFrame.</h2>

In [17]:
def load_96_well_plate_data(raw_data_file):
    '''Process raw data from a 96-well plate reader, resulting in a merged DataFrame in long format 
    with absorbance measurements at two different wavelengths (A750 and A415) using 'Well' as the index.'''
    
    def process_data(input_df, wavelength):
        '''Pivot the input DataFrame from wide to long format, generating a 'Well' column for the index.'''

        return (
            input_df
            .stack()
            .to_frame()
            .rename_axis(['Row', 'Column'], axis=0)
            .set_axis([wavelength], axis=1)
            .reset_index()
            .assign(Well = lambda df_: df_.Row + df_.Column)
            .set_index('Well')
            .drop(['Row', 'Column'], axis=1)
        )
    
    df750 = pd.read_excel(raw_data_file, sheet_name='750.0', header=1, index_col=0).pipe(process_data, 'A750')
    df415 = pd.read_excel(raw_data_file, sheet_name='415.0', header=1, index_col=0).pipe(process_data, 'A415')
    
    return pd.merge(df750, df415, on='Well')

<h3>Load the raw data from each 96-well plate into a DataFrame.
Store the DataFrames in a dictionary for easy access. Save in an Excel spreadsheet.</h3>

In [18]:
plate_dict = {}

for plate in plates_sheet_df.index:
    file = plates_sheet_df.loc[plate].File
    plate_dict[plate] = load_96_well_plate_data(raw_data_path/file)
    print(f'{plate}: {file}')
    plate_dict[plate].to_excel(plates_writer, sheet_name=plate)
    
plates_writer.close()

AB1: Plate AB1 (Serum Ab) 2021 05-20.xls
AB2: Plate AB2 (Serum Ab) 2021 05-20.xls
CD1: Plate CD1 (Serum Ab) 2021 05-20.xls
CD2: Plate CD2 (Serum Ab) 2021 05-20.xls
EF1: Plate EF1 (Serum Ab) 2021 05-20.xls
EF2: Plate EF2 (Serum Ab) 2021 05-20.xls
GH1: Plate GH1 (Serum Ab) 2021 05-20.xls
GH2: Plate GH2 (Serum Ab) 2021 05-20.xls


<h3>Make new columns that contain the absorbance measurements from the raw data files.</h3>

In [19]:
columns = ['Dilution_1_415',
           'Dilution_2_415',
           'Dilution_3_415',
           'Dilution_4_415',
           'Dilution_5_415',
           'Dilution_6_415',
           'Dilution_1_750',
           'Dilution_2_750',
           'Dilution_3_750',
           'Dilution_4_750',
           'Dilution_5_750',
           'Dilution_6_750',
           'Dilution_1_415_minus_750',
           'Dilution_2_415_minus_750',
           'Dilution_3_415_minus_750',
           'Dilution_4_415_minus_750',
           'Dilution_5_415_minus_750',
           'Dilution_6_415_minus_750']

wells = {'Dilution_1' : ('Dilution_1_415', 'Dilution_1_750', 'Dilution_1_415_minus_750'),
         'Dilution_2' : ('Dilution_2_415', 'Dilution_2_750', 'Dilution_2_415_minus_750'),
         'Dilution_3' : ('Dilution_3_415', 'Dilution_3_750', 'Dilution_3_415_minus_750'),
         'Dilution_4' : ('Dilution_4_415', 'Dilution_4_750', 'Dilution_4_415_minus_750'),
         'Dilution_5' : ('Dilution_5_415', 'Dilution_5_750', 'Dilution_5_415_minus_750'),
         'Dilution_6' : ('Dilution_6_415', 'Dilution_6_750', 'Dilution_6_415_minus_750')}


for column in columns:
    df[column] = np.NaN
    df[column].astype('float')

for index, row in df.iterrows():
    
    if row.Plate in plate_dict.keys():
        plate_df = plate_dict[row.Plate]
        
        for well in wells.keys():
            well_name = row[well]
            
            if well_name != 'None':
                col_415 = wells[well][0]
                col_750 = wells[well][1]
                col_415_minus_750 = wells[well][2]
                
                A415 = plate_df.loc[well_name].A415
                A750 = plate_df.loc[well_name].A750 
                
                #If value is off-scale, set to 4.0
                if A415 == '( + )':
                    A415 = 4.0
                                
                df.loc[index, col_415] = A415
                df.loc[index, col_750] = A750
                df.loc[index, col_415_minus_750] = A415 - A750
                

In [20]:
display_df_and_write_to_excel(df, writer, 'All Data');

,Group,#,Mouse,Plate,Dilution_1,Dilution_2,Dilution_3,Dilution_4,Dilution_5,Dilution_6,...,Dilution_3_750,Dilution_4_750,Dilution_5_750,Dilution_6_750,Dilution_1_415_minus_750,Dilution_2_415_minus_750,Dilution_3_415_minus_750,Dilution_4_415_minus_750,Dilution_5_415_minus_750,Dilution_6_415_minus_750
0,A,1,A1,AB1,A1,A2,A3,A4,A5,A6,...,0.047,0.046,0.048,0.047,0.085,0.044,0.025,0.021,0.017,0.018
1,A,2,A2,AB1,B1,B2,B3,B4,B5,B6,...,0.041,0.040,0.042,0.043,0.073,0.040,0.026,0.022,0.018,0.018
2,A,3,A3,AB1,C1,C2,C3,C4,C5,C6,...,0.046,0.044,0.045,0.046,0.064,0.036,0.022,0.018,0.017,0.016
3,A,4,A4,AB1,D1,D2,D3,D4,D5,D6,...,0.040,0.039,0.042,0.040,0.455,0.163,0.067,0.035,0.025,0.023
4,A,5,A5,AB1,E1,E2,E3,E4,E5,E6,...,0.037,0.037,0.036,0.037,1.031,0.344,0.108,0.047,0.033,0.027
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,H,4,H4,GH2,E12,E11,E10,E9,E8,E7,...,0.039,0.036,0.036,0.036,0.158,0.076,0.045,0.036,0.029,0.029
124,H,5,H5,GH2,D12,D11,D10,D9,D8,D7,...,0.041,0.039,0.043,0.040,0.377,0.183,0.077,0.041,0.026,0.026
125,H,6,H6,GH2,C12,C11,C10,C9,C8,C7,...,0.048,0.045,0.048,0.049,0.228,0.109,0.048,0.029,0.020,0.018
126,H,7,H7,GH2,B12,B11,B10,B9,B8,B7,...,0.043,0.041,0.043,0.045,0.237,0.168,0.115,0.081,0.040,0.022


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128 entries, 0 to 127
Data columns (total 28 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Group                     128 non-null    object 
 1   #                         128 non-null    int64  
 2   Mouse                     128 non-null    object 
 3   Plate                     128 non-null    object 
 4   Dilution_1                128 non-null    object 
 5   Dilution_2                128 non-null    object 
 6   Dilution_3                128 non-null    object 
 7   Dilution_4                128 non-null    object 
 8   Dilution_5                128 non-null    object 
 9   Dilution_6                128 non-null    object 
 10  Dilution_1_415            128 non-null    float64
 11  Dilution_2_415            128 non-null    float64
 12  Dilution_3_415            128 non-null    float64
 13  Dilution_4_415            128 non-null    float64
 14  Dilution_5

In [22]:
df.describe()

,#,Dilution_1_415,Dilution_2_415,Dilution_3_415,Dilution_4_415,Dilution_5_415,Dilution_6_415,Dilution_1_750,Dilution_2_750,Dilution_3_750,Dilution_4_750,Dilution_5_750,Dilution_6_750,Dilution_1_415_minus_750,Dilution_2_415_minus_750,Dilution_3_415_minus_750,Dilution_4_415_minus_750,Dilution_5_415_minus_750,Dilution_6_415_minus_750
count,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000
mean,4.500000,1.710758,1.118109,0.541258,0.219203,0.104117,0.074203,0.041664,0.041203,0.040727,0.040234,0.040727,0.040914,1.669094,1.076906,0.500531,0.178969,0.063391,0.033289
std,2.300291,1.626690,1.189683,0.643451,0.255807,0.069672,0.014952,0.004309,0.003728,0.003529,0.004807,0.004314,0.004467,1.627350,1.190031,0.643543,0.256068,0.069761,0.015425
min,1.000000,0.082000,0.077000,0.067000,0.062000,0.060000,0.061000,0.033000,0.035000,0.034000,0.033000,0.031000,0.032000,0.041000,0.036000,0.022000,0.018000,0.017000,0.016000
25%,2.750000,0.277500,0.149250,0.093750,0.074000,0.069000,0.066000,0.039000,0.039000,0.038000,0.037000,0.038000,0.038000,0.234750,0.103750,0.052500,0.034000,0.029000,0.025750
50%,4.500000,0.679500,0.371500,0.200000,0.110000,0.076000,0.070000,0.040000,0.040000,0.040000,0.039000,0.040000,0.040000,0.639500,0.331500,0.160500,0.070500,0.038500,0.031000
75%,6.250000,3.602500,2.350000,0.878000,0.263500,0.114000,0.077000,0.045250,0.043000,0.043000,0.042250,0.043250,0.045000,3.560500,2.314000,0.841250,0.227250,0.073250,0.037000
max,8.000000,4.000000,4.000000,3.335000,1.849000,0.573000,0.166000,0.051000,0.052000,0.048000,0.077000,0.052000,0.053000,3.967000,3.962000,3.295000,1.810000,0.533000,0.126000


In [23]:
name='Merged'

df_merged = (
    pd.merge(df, plates_sheet_df, left_on=['Plate'], right_on=['Plate'])
    .rename(columns={'Group_x':'Group'})
    .set_index(['Plate', 'Group', 'Antigen', 'Mouse'])
    .T
    .drop(['#', 'Dilution_1', 'Dilution_2', 'Dilution_3', 'Dilution_4',
       'Dilution_5', 'Dilution_6', 'File', 'Group_y', '1st Ab', '2nd Ab', 'Condition'])
    .pipe(display_df_and_write_to_excel, writer, sheet_name=name)
)

Plate                          AB1                                            \
Group                            A                                             
Antigen                  Antigen 1                                             
Mouse                           A1     A2     A3     A4     A5     A6     A7   
Dilution_1_415               0.136   0.12  0.112  0.496  1.069  0.132   0.12   
Dilution_2_415               0.092  0.081  0.082  0.204  0.384  0.089  0.092   
Dilution_3_415               0.072  0.067  0.068  0.107  0.145   0.07  0.076   
Dilution_4_415               0.067  0.062  0.062  0.074  0.084  0.111  0.069   
Dilution_5_415               0.065   0.06  0.062  0.067  0.069  0.062  0.069   
Dilution_6_415               0.065  0.061  0.062  0.063  0.064  0.065  0.069   
Dilution_1_750               0.051  0.047  0.048  0.041  0.038  0.036  0.038   
Dilution_2_750               0.048  0.041  0.046  0.041   0.04  0.037  0.039   
Dilution_3_750               0.047  0.041  0.046   0.04  0.037  0.037  0.039   
Dilution_4_750               0.046   0.04  0.044  0.039  0.037  0.077  0.038   
Dilution_5_750               0.048  0.042  0.045  0.042  0.036  0.036  0.039   
Dilution_6_750               0.047  0.043  0.046   0.04  0.037  0.038  0.039   
Dilution_1_415_minus_750     0.085  0.073  0.064  0.455  1.031  0.096  0.082   
Dilution_2_415_minus_750     0.044   0.04  0.036  0.163  0.344  0.052  0.053   
Dilution_3_415_minus_750     0.025  0.026  0.022  0.067  0.108  0.033  0.037   
Dilution_4_415_minus_750     0.021  0.022  0.018  0.035  0.047  0.034  0.031   
Dilution_5_415_minus_750     0.017  0.018  0.017  0.025  0.033  0.026   0.03   
Dilution_6_415_minus_750     0.018  0.018  0.016  0.023  0.027  0.027   0.03   

Plate                                             ...       GH2         \
Group                                   B         ...         G          
Antigen                         Antigen 1         ... Antigen 2          
Mouse                        A8        B1     B2  ...        G7     G8   
Dilution_1_415            1.674     0.136  0.082  ...     0.648  0.168   
Dilution_2_415            0.915     0.093  0.077  ...     0.268  0.107   
Dilution_3_415            0.319     0.076  0.075  ...     0.127   0.08   
Dilution_4_415            0.125     0.069  0.068  ...     0.084  0.071   
Dilution_5_415            0.079     0.068   0.07  ...     0.072  0.072   
Dilution_6_415            0.074     0.067  0.069  ...     0.074  0.088   
Dilution_1_750            0.039      0.04  0.041  ...     0.038  0.039   
Dilution_2_750            0.039      0.04  0.041  ...     0.039  0.039   
Dilution_3_750            0.039      0.04  0.042  ...     0.038  0.039   
Dilution_4_750             0.04     0.039  0.038  ...     0.037  0.039   
Dilution_5_750             0.04     0.039   0.04  ...     0.037  0.041   
Dilution_6_750            0.041     0.041  0.039  ...      0.04  0.053   
Dilution_1_415_minus_750  1.635     0.096  0.041  ...      0.61  0.129   
Dilution_2_415_minus_750  0.876     0.053  0.036  ...     0.229  0.068   
Dilution_3_415_minus_750   0.28     0.036  0.033  ...     0.089  0.041   
Dilution_4_415_minus_750  0.085      0.03   0.03  ...     0.047  0.032   
Dilution_5_415_minus_750  0.039     0.029   0.03  ...     0.035  0.031   
Dilution_6_415_minus_750  0.033     0.026   0.03  ...     0.034  0.035   

Plate                                                                         \
Group                            H                                             
Antigen                  Antigen 2                                             
Mouse                           H1     H2     H3     H4     H5     H6     H7   
Dilution_1_415               0.851  0.194  0.674  0.201  0.419  0.276  0.278   
Dilution_2_415               0.544  0.123  0.285  0.113  0.223  0.155   0.21   
Dilution_3_415               0.322  0.092  0.133  0.084  0.118  0.096  0.158   
Dilution_4_415               0.164  0.075  0.084  

In [24]:
name='Merged 415 nm'

df_merged_415 = (
    df_merged
    .drop(['Dilution_1_750', 'Dilution_2_750', 'Dilution_3_750',
                    'Dilution_4_750', 'Dilution_5_750', 'Dilution_6_750',
                    'Dilution_1_415_minus_750', 'Dilution_2_415_minus_750',
                    'Dilution_3_415_minus_750', 'Dilution_4_415_minus_750',
                    'Dilution_5_415_minus_750', 'Dilution_6_415_minus_750'])
    .assign(Dilution = list(dilutions_sheet_df.Dilution))
    .set_index('Dilution')
    .pipe(display_df_and_write_to_excel, writer, sheet_name=name)
)

Plate          AB1                                                             \
Group            A                                                          B   
Antigen  Antigen 1                                                  Antigen 1   
Mouse           A1     A2     A3     A4     A5     A6     A7     A8        B1   
Dilution                                                                        
200          0.136   0.12  0.112  0.496  1.069  0.132   0.12  1.674     0.136   
800          0.092  0.081  0.082  0.204  0.384  0.089  0.092  0.915     0.093   
3200         0.072  0.067  0.068  0.107  0.145   0.07  0.076  0.319     0.076   
12800        0.067  0.062  0.062  0.074  0.084  0.111  0.069  0.125     0.069   
51200        0.065   0.06  0.062  0.067  0.069  0.062  0.069  0.079     0.068   
204800       0.065  0.061  0.062  0.063  0.064  0.065  0.069  0.074     0.067   

Plate            ...       GH2                                               \
Group            ...         G                H                               
Antigen          ... Antigen 2        Antigen 2                               
Mouse        B2  ...        G7     G8        H1     H2     H3     H4     H5   
Dilution         ...                                                          
200       0.082  ...     0.648  0.168     0.851  0.194  0.674  0.201  0.419   
800       0.077  ...     0.268  0.107     0.544  0.123  0.285  0.113  0.223   
3200      0.075  ...     0.127   0.08     0.322  0.092  0.133  0.084  0.118   
12800     0.068  ...     0.084  0.071     0.164  0.075  0.084  0.072   0.08   
51200      0.07  ...     0.072  0.072     0.097  0.073   0.07  0.065  0.069   
204800    0.069  ...     0.074  0.088     0.077  0.073  0.065  0.065  0.066   

Plate                          
Group                          
Antigen                        
Mouse        H6     H7     H8  
Dilution                       
200       0.276  0.278  0.354  
800       0.155   0.21  0.195  
3200      0.096  0.158   0.11  
12800     0.074  0.122  0.081  
51200     0.068  0.083  0.072  
204800    0.067  0.067  0.068  

[6 rows x 128 columns]

In [25]:
name='Merged 750 nm'

df_merged_750 = (
    df_merged
    .drop(['Dilution_1_415', 'Dilution_2_415', 'Dilution_3_415',
                    'Dilution_4_415', 'Dilution_5_415', 'Dilution_6_415',
                    'Dilution_1_415_minus_750', 'Dilution_2_415_minus_750',
                    'Dilution_3_415_minus_750', 'Dilution_4_415_minus_750',
                    'Dilution_5_415_minus_750', 'Dilution_6_415_minus_750'])
    .assign(Dilution = list(dilutions_sheet_df.Dilution))
    .set_index('Dilution')
    .pipe(display_df_and_write_to_excel, writer, sheet_name=name)
)

Plate          AB1                                                             \
Group            A                                                          B   
Antigen  Antigen 1                                                  Antigen 1   
Mouse           A1     A2     A3     A4     A5     A6     A7     A8        B1   
Dilution                                                                        
200          0.051  0.047  0.048  0.041  0.038  0.036  0.038  0.039      0.04   
800          0.048  0.041  0.046  0.041   0.04  0.037  0.039  0.039      0.04   
3200         0.047  0.041  0.046   0.04  0.037  0.037  0.039  0.039      0.04   
12800        0.046   0.04  0.044  0.039  0.037  0.077  0.038   0.04     0.039   
51200        0.048  0.042  0.045  0.042  0.036  0.036  0.039   0.04     0.039   
204800       0.047  0.043  0.046   0.04  0.037  0.038  0.039  0.041     0.041   

Plate            ...       GH2                                               \
Group            ...         G                H                               
Antigen          ... Antigen 2        Antigen 2                               
Mouse        B2  ...        G7     G8        H1     H2     H3     H4     H5   
Dilution         ...                                                          
200       0.041  ...     0.038  0.039     0.039  0.041   0.04  0.043  0.042   
800       0.041  ...     0.039  0.039     0.039  0.041  0.038  0.037   0.04   
3200      0.042  ...     0.038  0.039     0.039  0.042  0.039  0.039  0.041   
12800     0.038  ...     0.037  0.039     0.039  0.039  0.038  0.036  0.039   
51200      0.04  ...     0.037  0.041     0.039  0.039  0.039  0.036  0.043   
204800    0.039  ...      0.04  0.053      0.04   0.04  0.037  0.036   0.04   

Plate                          
Group                          
Antigen                        
Mouse        H6     H7     H8  
Dilution                       
200       0.048  0.041  0.049  
800       0.046  0.042  0.047  
3200      0.048  0.043  0.046  
12800     0.045  0.041  0.046  
51200     0.048  0.043  0.048  
204800    0.049  0.045  0.049  

[6 rows x 128 columns]

In [26]:
name='Merged A415-A750'

df_merged_subtract = (
    df_merged
    .drop(['Dilution_1_415', 'Dilution_2_415', 'Dilution_3_415',
                    'Dilution_4_415', 'Dilution_5_415', 'Dilution_6_415',
                         'Dilution_1_750', 'Dilution_2_750', 'Dilution_3_750',
                         'Dilution_4_750', 'Dilution_5_750', 'Dilution_6_750'])
    .assign(Dilution = list(dilutions_sheet_df.Dilution))
    .set_index('Dilution')
    .pipe(display_df_and_write_to_excel, writer, sheet_name=name)
)

Plate          AB1                                                             \
Group            A                                                          B   
Antigen  Antigen 1                                                  Antigen 1   
Mouse           A1     A2     A3     A4     A5     A6     A7     A8        B1   
Dilution                                                                        
200          0.085  0.073  0.064  0.455  1.031  0.096  0.082  1.635     0.096   
800          0.044   0.04  0.036  0.163  0.344  0.052  0.053  0.876     0.053   
3200         0.025  0.026  0.022  0.067  0.108  0.033  0.037   0.28     0.036   
12800        0.021  0.022  0.018  0.035  0.047  0.034  0.031  0.085      0.03   
51200        0.017  0.018  0.017  0.025  0.033  0.026   0.03  0.039     0.029   
204800       0.018  0.018  0.016  0.023  0.027  0.027   0.03  0.033     0.026   

Plate            ...       GH2                                               \
Group            ...         G                H                               
Antigen          ... Antigen 2        Antigen 2                               
Mouse        B2  ...        G7     G8        H1     H2     H3     H4     H5   
Dilution         ...                                                          
200       0.041  ...      0.61  0.129     0.812  0.153  0.634  0.158  0.377   
800       0.036  ...     0.229  0.068     0.505  0.082  0.247  0.076  0.183   
3200      0.033  ...     0.089  0.041     0.283   0.05  0.094  0.045  0.077   
12800      0.03  ...     0.047  0.032     0.125  0.036  0.046  0.036  0.041   
51200      0.03  ...     0.035  0.031     0.058  0.034  0.031  0.029  0.026   
204800     0.03  ...     0.034  0.035     0.037  0.033  0.028  0.029  0.026   

Plate                          
Group                          
Antigen                        
Mouse        H6     H7     H8  
Dilution                       
200       0.228  0.237  0.305  
800       0.109  0.168  0.148  
3200      0.048  0.115  0.064  
12800     0.029  0.081  0.035  
51200      0.02   0.04  0.024  
204800    0.018  0.022  0.019  

[6 rows x 128 columns]

In [27]:
def format_altair_chart_dataframe(df, value_name):
        
    formatted_df = (
        df
        .melt(value_name=value_name, ignore_index=False)
        .assign(**{value_name: lambda df_:df_[value_name].astype(float)})
        .assign(**{f'Log_{value_name}': lambda df_: np.log10(df_[value_name])})
        .reset_index()
        .assign(Log_Dilution = lambda df_: np.log10(df_.Dilution))
        .set_index(['Plate', 'Group', 'Antigen', 'Mouse', 'Dilution'])
    )
                
    return formatted_df

In [28]:
def plot_altair_groups_x_antigens(df, x_name, y_name, rules, y_scale):
    
    groups = np.unique(df.index.get_level_values(1))
    antigens = np.unique(df.index.get_level_values(2))

    column_index = pd.IndexSlice[:]

    group_charts = []

    highlight = alt.selection(type='single', on='mousedown', fields=['Mouse'])
    color = alt.condition(highlight,
                          alt.Color('Mouse:N', legend=None),
                          alt.value('lightgray'))

    for group in groups:

        antigen_charts = []

        for antigen in antigens:
            group_antigen_index = pd.IndexSlice[:, group, antigen, :, :]
            
            try:
                gr_ag_df = df.loc[group_antigen_index, column_index].reset_index()
            except KeyError as exp:
                print(f'{type(exp)}: {exp}')
                gr_ag_df = pd.DataFrame()

            base = alt.Chart(gr_ag_df).encode(
                alt.X(x_name, type='quantitative'),
                alt.Y(y_name, type='quantitative', scale=y_scale),
                color=color
                ).properties(
                    width=150,
                    height=150,
                    title=antigen
                )

            scatter = base.mark_point(filled=True).encode()
            line = base.mark_line()
            
            chart = line + scatter
            
            for r in rules:
                chart += alt.Chart(pd.DataFrame({'y': [r]})).mark_rule(color='red').encode(y='y')


            antigen_charts.append(chart)


        legend = alt.Chart(gr_ag_df).mark_point(filled=True, size=120).encode(
            y=alt.Y('Mouse', type='nominal', axis=alt.Axis(orient='right')),
            color=color).add_selection(highlight)
        antigen_charts.append(legend)

        group_charts.append(alt.hconcat(*antigen_charts))


    return alt.vconcat(*group_charts)

In [29]:
A415_minus_A750_chart_data_df = format_altair_chart_dataframe(df_merged_subtract, 'A415_minus_A750')
A415_minus_A750_chart_data_df

A415_minus_A750  Log_A415_minus_A750  \
Plate Group Antigen   Mouse Dilution                                         
AB1   A     Antigen 1 A1    200                 0.085            -1.070581   
                            800                 0.044            -1.356547   
                            3200                0.025            -1.602060   
                            12800               0.021            -1.677781   
                            51200               0.017            -1.769551   
...                                               ...                  ...   
GH2   H     Antigen 2 H8    800                 0.148            -0.829738   
                            3200                0.064            -1.193820   
                            12800               0.035            -1.455932   
                            51200               0.024            -1.619789   
                            204800              0.019            -1.721246   

                                      Log_Dilution  
Plate Group Antigen   Mouse Dilution                
AB1   A     Antigen 1 A1    200            2.30103  
                            800            2.90309  
                            3200           3.50515  
                            12800          4.10721  
                            51200          4.70927  
...                                            ...  
GH2   H     Antigen 2 H8    800            2.90309  
                            3200           3.50515  
                            12800          4.10721  
                            51200          4.70927  
                            204800         5.31133  

[768 rows x 3 columns]

In [30]:
plots = plot_altair_groups_x_antigens(A415_minus_A750_chart_data_df, 'Log_Dilution', 'Log_A415_minus_A750',
                               [np.log10(CUTOFF)],
                               alt.Scale(domain=(-2, 0.7), clamp=True))
plots.save(output_path / f'{EXPERIMENT_NAME.name} groups x antigens (individual mice) Log A415_minus_A750 {todays_date}.html',
           embed_options={'renderer':'svg'})

In [31]:
def CalculateTiters(group, mouse, antigen, xyc_df):
    
    # xyc_df must contain x, y, and at least one cutoff column
    calc_df = xyc_df.copy()
    
    cutoffs = calc_df.columns[calc_df.columns.str.contains('cutoff')]
    titers = cutoffs.str.replace('cutoff', 'titer')

    # Titers will be added to the results_df when they are calculated
    
    results_df = pd.DataFrame({'Group': group,'Mouse': mouse,'Antigen': antigen}, index=pd.Index([0]))
    
    for t, c in zip(titers, cutoffs):

        calc_df['delta'] = calc_df.y - calc_df[c]
        calc_df['above_cutoff'] = calc_df.delta > 0
        calc_df['diff_'] = calc_df.above_cutoff.astype(int).diff()
        
        # Set titer to lowest value.
        
        # If the first dilution is above the cutoff, calcualte titer in the usual way
        # (i.e the highest titer that is above the baseline). Otherwise, the titer is the lowest value.
        
        titer = calc_df.x[0]
        if calc_df.above_cutoff[0]:

            # This mask will select the rows where the measurement changes from
            # above the cutoff to below the cutoff. We then need to find the lowest
            # index (i.e. the first crossover point) and subtract one to obtain the 
            # highest measurement above the cutoff.
            
            calc_df_dropna = calc_df.dropna(subset=['diff_'])
            crossover_mask = (calc_df_dropna.diff_ == -1)
            # this indicates that this row is below the cutoff and the previous row is above

            num_crossovers = sum(crossover_mask)

            if num_crossovers > 0:
                row_index = calc_df_dropna[crossover_mask].index[0]
                if row_index > 0:
                    row_index -= 1
                titer = calc_df.loc[row_index, :].x.astype(float)
            else:
                last_element = len(calc_df.x) - 1
                titer = calc_df.x[last_element]
                              
        results_df[t] = titer
      
    return results_df

In [32]:
def plot_group_for_each_antigen(input_df, sham_mice, file_label, y_label, y_limits, cutoff, antigens=None):       
    
    # This dataframe will hold the calculations to be exported
    all_titers_df = pd.DataFrame(columns=['Group', 'Mouse', 'Antigen', 'titer'], index=pd.Index([0]))

    # Structure of the Dataframe df_merged_415
    #   Column Index: Plate, Group, Antigen, Mouse
    #   Row Index: Dilution

    if antigens == None:
        antigens = input_df.T.reset_index().Antigen.unique()
        
    for antigen in antigens:

        row_index = pd.IndexSlice[:] # get all rows
        sham_index = pd.IndexSlice[:, 'A', antigen, sham_mice] # select just the indicated sham mice in Group A for one antigen
        all_mice_index = pd.IndexSlice[:, :, antigen, :] # select all mice for one antigen

        
        temp_df = (
            input_df
            .loc[row_index, sham_index] 
        )
        
        df_by_antigen_sham = (
            temp_df
            .assign(**{'Mean':lambda df_:temp_df.mean(axis=1),
                       'SD':lambda df_:temp_df.std(axis=1, ddof=1),
                       'Mean_Plus_3xSD':lambda df_:df_.Mean + (3*df_.SD),
                       f'cutoff={cutoff}': lambda df_:np.maximum(df_.Mean_Plus_3xSD, cutoff)}
                   )
            .pipe(display_df_and_write_to_excel, writer, sheet_name=f'{antigen} (Sham)')
        )
        
        df_by_antigen_all_mice = (
            input_df
            .loc[row_index, all_mice_index]
            .pipe(display_df_and_write_to_excel, writer, sheet_name=antigen)
       )

        # Generate many interpolated values between the high and low dilutions
        # Plot log10 dilution vs. log10 A415

        sham_baseline_1 = df_by_antigen_sham[f'cutoff={cutoff}']
        sham_baseline_2 = df_by_antigen_sham.Mean_Plus_3xSD
        sham_baseline_3 = df_by_antigen_sham.Mean

        x = np.log10(sham_baseline_1.index)
        low_value = x[0]
        high_value = x[-1] + 0.5 #(extrapolation)
        x_interpolated = np.linspace(low_value, high_value,1000)

        sham_y1 = np.log10(sham_baseline_1)
        sham_y2 = np.log10(sham_baseline_2)
        sham_y3 = np.log10(sham_baseline_3)

        sham_interpolation_1 = sp.interpolate.interp1d(x, sham_y1, kind = 'linear', fill_value = 'extrapolate')
        sham_interpolation_2 = sp.interpolate.interp1d(x, sham_y2, kind = 'linear', fill_value = 'extrapolate')
        sham_interpolation_3 = sp.interpolate.interp1d(x, sham_y3, kind = 'linear', fill_value = 'extrapolate')

        sham_y1_interpolated = sham_interpolation_1(x_interpolated)
        sham_y2_interpolated = sham_interpolation_2(x_interpolated)
        sham_y3_interpolated = sham_interpolation_3(x_interpolated)


        # Plot one mouse per graph with all mice in a group in one figure

        groups = df_by_antigen_all_mice.columns.get_level_values(1).unique()
        groups = list(groups)

        for group in groups:

            mice_in_group = pd.IndexSlice[:, group, antigen, :]
            mice = list(df_by_antigen_all_mice.loc[row_index, mice_in_group].columns.get_level_values(3).unique())

            num_mice = len(mice)

            num_columns = 4
            num_rows = num_mice // num_columns
            if num_mice % num_columns:
                num_rows += 1

            if num_rows == 1:
                num_rows = 2


            fig, axes = plt.subplots(num_rows, num_columns, figsize=(num_columns*4,num_rows*5), sharex=True, sharey=True)
            axes[0, 0].set_xlim([low_value-0.3, high_value+0.2])
            axes[0, 0].set_ylim(y_limits)
            fig.suptitle(f'{EXPERIMENT_NAME.name}\n{antigen}: (Group {group})\n{todays_date}', fontsize=14, fontweight='bold')      

            for i, mouse in enumerate(mice):

                column_index = pd.IndexSlice[:, :, antigen, mouse]

                y = np.log10(list(df_by_antigen_all_mice.loc[row_index, column_index].values.ravel()))

                mouse_interpolation = sp.interpolate.interp1d(x, y, kind = 'linear', fill_value= 'extrapolate')
                mouse_y_interpolated = mouse_interpolation(x_interpolated)


                # ====================================================================================
                titers_df = CalculateTiters(group, mouse, antigen,
                                            pd.DataFrame({'x': x_interpolated, 'y': mouse_y_interpolated,
                                                          'cutoff': sham_y1_interpolated}))
                
                all_titers_df = pd.concat([all_titers_df, titers_df])

                titer = titers_df.titer[0]
                # ====================================================================================


                row = i // num_columns
                column = i % num_columns
                if column == 0:
                    axes[row, column].set_ylabel(y_label)

                line_1 = axes[row, column].plot(x_interpolated, sham_y1_interpolated, '-', linewidth=1, color='blue')[0]
                line_2 = axes[row, column].plot(x_interpolated, sham_y2_interpolated, '-', linewidth=1, color='red')[0]
                line_3 = axes[row, column].plot(x_interpolated, mouse_y_interpolated, '-', linewidth=3, color='black')[0]
                line_4 = axes[row, column].plot(x_interpolated, sham_y3_interpolated, '-', linewidth=1, color='purple')[0]

                axes[row, column].plot(x, sham_y1, 'ob',
                                       x, sham_y2, 'or',
                                       x, y, 'ok',
                                       x, sham_y3, 'om')

                axes[row, column].set_title(f'Mouse: {mouse}')
                axes[row, column].axvline(x=titer, color='b')
             
                x_annotate = 0.1
                y_annotate = -0.1

                axes[row, column].annotate(f'Titer = {titer:.2f}', xy=(titer+x_annotate, y_annotate), rotation=90, fontsize=12, color='b')
                axes[row, column].set_xlabel('Log10 Serum Dilution')

                axes[row, column].fill_between(x_interpolated, sham_y3_interpolated, sham_y1_interpolated, color='blue', alpha=0.2)
                axes[row, column].fill_between(x_interpolated, sham_y3_interpolated, sham_y2_interpolated, color='red', alpha=0.2)
                axes[row, column].fill_between(x_interpolated, sham_y3_interpolated, mouse_y_interpolated, color='black', alpha=0.3)

                labels = [f'cutoff={cutoff} or (Sham Mean + 3xSD)',
                          'Sham Mean + 3xSD',
                          'Mouse',
                          'Sham Mean']*2


            fig.legend(handles=[line_1, line_2, line_3, line_4],     # The line objects
                           labels=labels,        # The labels for each line
                           loc="center right",   # Position of legend
                           borderaxespad=0.2,    # Small spacing around legend box
                           title=f'{antigen}: (Group {group}) [{file_label}]')      # Title for the legend

            # Adjust the scaling factor to fit your legend text completely outside the plot
            # (smaller value results in more space being made for the legend)
            
            plt.subplots_adjust(wspace=0.1, hspace=0.2, right=0.8)

            file_name = output_path / f'{antigen} - (Group {group}) [{file_label}] {todays_date}.png'
            plt.savefig(file_name, bbox_inches='tight')
            plt.close(fig)
            print(file_name)
        
    return all_titers_df.copy().dropna().reset_index(drop=True)

In [33]:
def save_titers(all_titers_df, name):
    all_titers_df.to_excel(writer, sheet_name=f'All Titers ({name})')
    all_titers_df = all_titers_df.set_index(['Group', 'Antigen', 'Mouse']).T
    all_titers_df.to_excel(writer, sheet_name=f"All Titers 2 ({name})")

    transposed_df = (
        all_titers_df
        .stack()
        .T
        .reset_index()
        .set_index(['Antigen', 'Group'])
        .pipe(display_df_and_write_to_excel, writer, sheet_name=f'All Titers 3 ({name})')
    )


In [34]:
sham_df = df_merged_subtract.copy().T.reset_index()
SHAM_MICE = list(sham_df[sham_df.Group == 'A'].Mouse.unique())

if EXPERIMENT_NAME == ExperimentName.Bp_m01:
    # For Bp m01, A4, A5, and A8 reacted to HI-LVS capB, HI-Bp82, and Hcp2
    # These three mice were excluded from calculations of the mean and SD for all antigens
    SHAM_MICE.remove('A4')
    SHAM_MICE.remove('A5')
    SHAM_MICE.remove('A8')
elif EXPERIMENT_NAME == ExperimentName.Bp_m02:
    # For Bp m02, A4 and A7 reacted to HI-LVS capB, HI-Bp82, and Hcp2
    # These mice were excluded from calculations of the mean and SD for all antigens
    SHAM_MICE.remove('A4')
    SHAM_MICE.remove('A7')
elif EXPERIMENT_NAME == ExperimentName.EXAMPLE:
    SHAM_MICE.remove('A4')
    SHAM_MICE.remove('A5')
    SHAM_MICE.remove('A8')
    
    
    
print(f'{SHAM_MICE=}')


all_titers_df = plot_group_for_each_antigen(df_merged_subtract,
                                            SHAM_MICE,
                                            'A415-A750',
                                            'Log (A415 - A750)',
                                            [-1.8, 0.7],
                                            cutoff=CUTOFF)

SHAM_MICE=['A1', 'A2', 'A3', 'A6', 'A7']


Plate          AB1                                Mean        SD  \
Group            A                                                 
Antigen  Antigen 1                                                 
Mouse           A1     A2     A3     A6     A7                     
Dilution                                                           
200          0.085  0.073  0.064  0.096  0.082  0.0800  0.012145   
800          0.044   0.04  0.036  0.052  0.053  0.0450  0.007416   
3200         0.025  0.026  0.022  0.033  0.037  0.0286  0.006189   
12800        0.021  0.022  0.018  0.034  0.031  0.0252  0.006907   
51200        0.017  0.018  0.017  0.026   0.03  0.0216  0.006025   
204800       0.018  0.018  0.016  0.027   0.03  0.0218  0.006261   

Plate    Mean_Plus_3xSD cutoff=0.05  
Group                                
Antigen                              
Mouse                                
Dilution                             
200            0.116435    0.116435  
800            0.067249    0.067249  
3200           0.047166    0.050000  
12800          0.045920    0.050000  
51200          0.039675    0.050000  
204800         0.040583    0.050000

Plate          AB1                                                             \
Group            A                                                          B   
Antigen  Antigen 1                                                  Antigen 1   
Mouse           A1     A2     A3     A4     A5     A6     A7     A8        B1   
Dilution                                                                        
200          0.085  0.073  0.064  0.455  1.031  0.096  0.082  1.635     0.096   
800          0.044   0.04  0.036  0.163  0.344  0.052  0.053  0.876     0.053   
3200         0.025  0.026  0.022  0.067  0.108  0.033  0.037   0.28     0.036   
12800        0.021  0.022  0.018  0.035  0.047  0.034  0.031  0.085      0.03   
51200        0.017  0.018  0.017  0.025  0.033  0.026   0.03  0.039     0.029   
204800       0.018  0.018  0.016  0.023  0.027  0.027   0.03  0.033     0.026   

Plate            ...       GH1                                               \
Group            ...         G                H                               
Antigen          ... Antigen 1        Antigen 1                               
Mouse        B2  ...        G7     G8        H1     H2     H3     H4     H5   
Dilution         ...                                                          
200       0.041  ...     3.034  3.961     3.091  2.154  3.959   2.07  3.098   
800       0.036  ...      1.58  3.123     1.945  0.991  3.962  0.924   1.89   
3200      0.033  ...     0.604  1.499     0.737  0.288  2.657  0.222   0.75   
12800      0.03  ...      0.17  0.513     0.223  0.091  1.298  0.067  0.216   
51200      0.03  ...     0.058  0.138     0.074  0.043  0.412  0.033  0.068   
204800     0.03  ...     0.039  0.053     0.035  0.033  0.114  0.028  0.032   

Plate                          
Group                          
Antigen                        
Mouse        H6     H7     H8  
Dilution                       
200        3.95  3.957  3.949  
800       2.557  3.209  2.471  
3200       1.23   2.01  1.025  
12800     0.387  0.785  0.279  
51200     0.109   0.23  0.069  
204800    0.035  0.069  0.023  

[6 rows x 64 columns]

C:\Users\mitullius\Notebook\2023\Python\serum-antibody-ELISA\example\output 2023-02-27\Antigen 1 - (Group A) [A415-A750] 2023-02-27.png
C:\Users\mitullius\Notebook\2023\Python\serum-antibody-ELISA\example\output 2023-02-27\Antigen 1 - (Group B) [A415-A750] 2023-02-27.png
C:\Users\mitullius\Notebook\2023\Python\serum-antibody-ELISA\example\output 2023-02-27\Antigen 1 - (Group C) [A415-A750] 2023-02-27.png
C:\Users\mitullius\Notebook\2023\Python\serum-antibody-ELISA\example\output 2023-02-27\Antigen 1 - (Group D) [A415-A750] 2023-02-27.png
C:\Users\mitullius\Notebook\2023\Python\serum-antibody-ELISA\example\output 2023-02-27\Antigen 1 - (Group E) [A415-A750] 2023-02-27.png
C:\Users\mitullius\Notebook\2023\Python\serum-antibody-ELISA\example\output 2023-02-27\Antigen 1 - (Group F) [A415-A750] 2023-02-27.png
C:\Users\mitullius\Notebook\2023\Python\serum-antibody-ELISA\example\output 2023-02-27\Antigen 1 - (Group G) [A415-A750] 2023-02-27.png
C:\Users\mitullius\Notebook\2023\Python\serum-an

Plate          AB2                                Mean        SD  \
Group            A                                                 
Antigen  Antigen 2                                                 
Mouse           A1     A2     A3     A6     A7                     
Dilution                                                           
200          0.092  0.084  0.094  0.095  0.094  0.0918  0.004494   
800          0.046  0.042  0.043  0.049  0.051  0.0462  0.003834   
3200         0.031  0.029  0.028  0.035   0.04  0.0326  0.004930   
12800        0.026  0.028  0.024  0.029  0.034  0.0282  0.003768   
51200        0.022  0.023  0.021  0.028  0.033  0.0254  0.005030   
204800       0.022  0.023   0.02  0.027  0.032  0.0248  0.004764   

Plate    Mean_Plus_3xSD cutoff=0.05  
Group                                
Antigen                              
Mouse                                
Dilution                             
200            0.105283    0.105283  
800            0.057702    0.057702  
3200           0.047389    0.050000  
12800          0.039505    0.050000  
51200          0.040490    0.050000  
204800         0.039093    0.050000

Plate          AB2                                                             \
Group            A                                                          B   
Antigen  Antigen 2                                                  Antigen 2   
Mouse           A1     A2     A3     A4     A5     A6     A7     A8        B1   
Dilution                                                                        
200          0.092  0.084  0.094   0.28   0.15  0.095  0.094  0.405     3.431   
800          0.046  0.042  0.043  0.102  0.078  0.049  0.051  0.279     1.969   
3200         0.031  0.029  0.028  0.051  0.045  0.035   0.04  0.171     0.686   
12800        0.026  0.028  0.024  0.033  0.032  0.029  0.034  0.082     0.183   
51200        0.022  0.023  0.021  0.028  0.031  0.028  0.033  0.044     0.062   
204800       0.022  0.023   0.02  0.028  0.027  0.027  0.032  0.037     0.036   

Plate            ...       GH2                                               \
Group            ...         G                H                               
Antigen          ... Antigen 2        Antigen 2                               
Mouse        B2  ...        G7     G8        H1     H2     H3     H4     H5   
Dilution         ...                                                          
200       0.182  ...      0.61  0.129     0.812  0.153  0.634  0.158  0.377   
800       0.074  ...     0.229  0.068     0.505  0.082  0.247  0.076  0.183   
3200      0.042  ...     0.089  0.041     0.283   0.05  0.094  0.045  0.077   
12800     0.034  ...     0.047  0.032     0.125  0.036  0.046  0.036  0.041   
51200     0.032  ...     0.035  0.031     0.058  0.034  0.031  0.029  0.026   
204800    0.032  ...     0.034  0.035     0.037  0.033  0.028  0.029  0.026   

Plate                          
Group                          
Antigen                        
Mouse        H6     H7     H8  
Dilution                       
200       0.228  0.237  0.305  
800       0.109  0.168  0.148  
3200      0.048  0.115  0.064  
12800     0.029  0.081  0.035  
51200      0.02   0.04  0.024  
204800    0.018  0.022  0.019  

[6 rows x 64 columns]

C:\Users\mitullius\Notebook\2023\Python\serum-antibody-ELISA\example\output 2023-02-27\Antigen 2 - (Group A) [A415-A750] 2023-02-27.png
C:\Users\mitullius\Notebook\2023\Python\serum-antibody-ELISA\example\output 2023-02-27\Antigen 2 - (Group B) [A415-A750] 2023-02-27.png
C:\Users\mitullius\Notebook\2023\Python\serum-antibody-ELISA\example\output 2023-02-27\Antigen 2 - (Group C) [A415-A750] 2023-02-27.png
C:\Users\mitullius\Notebook\2023\Python\serum-antibody-ELISA\example\output 2023-02-27\Antigen 2 - (Group D) [A415-A750] 2023-02-27.png
C:\Users\mitullius\Notebook\2023\Python\serum-antibody-ELISA\example\output 2023-02-27\Antigen 2 - (Group E) [A415-A750] 2023-02-27.png
C:\Users\mitullius\Notebook\2023\Python\serum-antibody-ELISA\example\output 2023-02-27\Antigen 2 - (Group F) [A415-A750] 2023-02-27.png
C:\Users\mitullius\Notebook\2023\Python\serum-antibody-ELISA\example\output 2023-02-27\Antigen 2 - (Group G) [A415-A750] 2023-02-27.png
C:\Users\mitullius\Notebook\2023\Python\serum-an

In [35]:
save_titers(all_titers_df, 'A415-A750')

titer                                                 \
Mouse                 A1       A2       A3        A4        A5       A6   
Antigen   Group                                                           
Antigen 1 A      2.30103  2.30103  2.30103  3.773318  4.061451  2.30103   
Antigen 2 A      2.30103  2.30103  2.30103  3.530865  3.348146  2.30103   
Antigen 1 B          NaN      NaN      NaN       NaN       NaN      NaN   
Antigen 2 B          NaN      NaN      NaN       NaN       NaN      NaN   
Antigen 1 C          NaN      NaN      NaN       NaN       NaN      NaN   
Antigen 2 C          NaN      NaN      NaN       NaN       NaN      NaN   
Antigen 1 D          NaN      NaN      NaN       NaN       NaN      NaN   
Antigen 2 D          NaN      NaN      NaN       NaN       NaN      NaN   
Antigen 1 E          NaN      NaN      NaN       NaN       NaN      NaN   
Antigen 2 E          NaN      NaN      NaN       NaN       NaN      NaN   
Antigen 1 F          NaN      NaN      NaN       NaN       NaN      NaN   
Antigen 2 F          NaN      NaN      NaN       NaN       NaN      NaN   
Antigen 1 G          NaN      NaN      NaN       NaN       NaN      NaN   
Antigen 2 G          NaN      NaN      NaN       NaN       NaN      NaN   
Antigen 1 H          NaN      NaN      NaN       NaN       NaN      NaN   
Antigen 2 H          NaN      NaN      NaN       NaN       NaN      NaN   

                                                        ...            \
Mouse                 A7        A8        B1        B2  ...        G7   
Antigen   Group                                         ...             
Antigen 1 A      2.30103  4.514733       NaN       NaN  ...       NaN   
Antigen 2 A      2.30103  4.585009       NaN       NaN  ...       NaN   
Antigen 1 B          NaN       NaN  2.301030  2.301030  ...       NaN   
Antigen 2 B          NaN       NaN  4.946932  3.256787  ...       NaN   
Antigen 1 C          NaN       NaN       NaN       NaN  ...       NaN   
Antigen 2 C          NaN       NaN       NaN       NaN  ...       NaN   
Antigen 1 D          NaN       NaN       NaN       NaN  ...       NaN   
Antigen 2 D          NaN       NaN       NaN       NaN  ...       NaN   
Antigen 1 E          NaN       NaN       NaN       NaN  ...       NaN   
Antigen 2 E          NaN       NaN       NaN       NaN  ...       NaN   
Antigen 1 F          NaN       NaN       NaN       NaN  ...       NaN   
Antigen 2 F          NaN       NaN       NaN       NaN  ...       NaN   
Antigen 1 G          NaN       NaN       NaN       NaN  ...  4.932877   
Antigen 2 G          NaN       NaN       NaN       NaN  ...  4.047395   
Antigen 1 H          NaN       NaN       NaN       NaN  ...       NaN   
Antigen 2 H          NaN       NaN       NaN       NaN  ...       NaN   

                                                                             \
Mouse                  G8        H1        H2        H3        H4        H5   
Antigen   Group                                                               
Antigen 1 A           NaN       NaN       NaN       NaN       NaN       NaN   
Antigen 2 A           NaN       NaN       NaN       NaN       NaN       NaN   
Antigen 1 B           NaN       NaN       NaN       NaN       NaN       NaN   
Antigen 2 B           NaN       NaN       NaN       NaN       NaN       NaN   
Antigen 1 C           NaN       NaN       NaN       NaN       NaN       NaN   
Antigen 2 C           NaN       NaN       NaN       NaN       NaN       NaN   
Antigen 1 D           NaN       NaN       NaN       NaN       NaN       NaN   
Antigen 2 D           NaN       NaN       NaN       NaN       NaN       NaN   
Antigen 1 E           NaN       NaN       NaN       NaN       NaN       NaN   
Antigen 2 E           NaN       NaN       NaN       NaN       NaN       NaN   
Antigen 1 F           NaN       NaN       NaN       NaN       NaN       NaN   
Antigen 2 F           NaN       NaN       NaN       NaN       NaN       NaN   
Antigen 1 G      5.3

In [36]:
writer.close()